In [ ]:
import xarray as xr
import numpy as np
from math import pi
import datetime as dt
import os
import sys
from os.path import exists
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import *; from dateutil.relativedelta import *
from scipy.interpolate import make_interp_spline, BSpline
from scipy.signal import savgol_filter
import scipy.ndimage as ndimage
import cartopy.crs as ccrs
import cartopy as cart
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
adir_figs='F:/data/NASA_biophysical/biforcation_figures_chelle/figs/'
adir_data='F:/data/NASA_biophysical/biforcation_figures_chelle/'

#ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon').sortby('lat').drop({'year'})
def data_smooth(U):
    z.load()
    U=z.copy()
    V=U.copy()
    V=V.fillna(0)
    VV=ndimage.gaussian_filter(V,sigma=5.0,order=0)

    W=0*U.copy()+1
    W=W.fillna(0)
    #W[np.isnan(U)]=0
    WW=ndimage.gaussian_filter(W,sigma=5.0,order=0)

    Z=VV/WW
    return Z

In [ ]:
#dir_data_oscar = 'F:/data/sat_data/oscar/L4/oscar_third_deg/oscar_vel*.nc'
#ds=xr.open_mfdataset(dir_data_oscar,combine='nested',concat_dim='time').isel(depth=0).drop({'um','vm'}).rename({'latitude':'lat','longitude':'lon'})
#ds = ds.sel(lon=slice(20.0,379.9))
#ds = ds.sortby('lat').drop({'year'})


In [ ]:
filelist=[]
dir_data = 'F:/data/sat_data/aviso/'
from pathlib import Path
for filename in Path(dir_data).rglob('*.nc'):
    filelist.append(filename)
ds=xr.open_mfdataset(filelist,combine='nested',concat_dim='time').drop({'ugosa','vgosa','err'}).rename({'latitude':'lat','longitude':'lon'})
#ds=xr.open_dataset(filelist[0]) #,combine='nested',concat_dim='time').isel(depth=0).drop({'um','vm'}).rename({'latitude':'lat','longitude':'lon'})
#ds.adt[0,:,:].plot()

In [ ]:
#ds=xr.open_mfdataset(filelist,combine='nested',concat_dim='time').drop({'ugosa','vgosa','err'}).rename({'latitude':'lat','longitude':'lon'})
#ds=xr.open_dataset(filelist[0]) #,combine='nested',concat_dim='time').isel(depth=0).drop({'um','vm'}).rename({'latitude':'lat','longitude':'lon'})
ds.adt[0,:,:].plot()

In [ ]:
ds2 = ds.resample(time='QS-DEC',keep_attrs=True).mean(skipna=False)
ds2 = ds2.sel(lon=slice(100,260),lat=slice(20,65))
X,Y=np.meshgrid(ds2.lon,ds2.lat)
#plt.figure()
#z=ds2.adt[0,:,:]
#z2=ndimage.gaussian_filter(z,sigma=5,order=0)
#z3=data_smooth(z)
#CS = plt.contour(X,Y,z3,30)

In [ ]:
maxcon2=[]
plt.figure(figsize=(15,6))
for itime in range(ds2.time.size):
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
    ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')
    ax.coastlines()
    X,Y=np.meshgrid(ds2.lon,ds2.lat)
    ds2 = ds2.sel(lon=slice(100,260),lat=slice(20,65))
    z=ds2.adt[itime,:,:]
    z3=data_smooth(z)
    z3[np.isnan(z)]=np.nan
    CS = ax.contour(X,Y,z3,30)
    num_levels = len(CS.allsegs)
    maxlat=40
    for ilev in range(num_levels):
        num_segments = len(CS.allsegs[ilev])
        index_save,jndex_save=[],[]
        for iseg in range(num_segments):
            num_pts = len(CS.allsegs[ilev][iseg])
            aseg = CS.allsegs[ilev][iseg]
            ax.plot(aseg[:,0],aseg[:,1],color='grey',transform=ccrs.PlateCarree())
            if itime==8 and max(aseg[:,1])>45:  #this is to fix a broken contour
                continue
            if itime==60 and aseg[1,1]>40:  #this is to fix a broken contour
                continue
            if itime==93 and aseg[1,1]>45:  #this is to fix a broken contour
                continue
            if itime==100 and aseg[1,1]>45:  #this is to fix a broken contour
                continue
            if min(aseg[:,0])>180 or max(aseg[:,1])>55:
                continue
            ind = (aseg[:,0]>235) & (aseg[:,0]<240) & (aseg[:,1]<48)
            if any(ind) & (aseg[:,1].max()<51):
                index_save.append(ilev)            
                jndex_save.append(iseg)            
            if any(ind) & any(aseg[:,1]>maxlat) & any(aseg[:,0]<230):
                maxlat=aseg[:,1].max().data
                maxcon=aseg
                print(aseg[1,:])
    ax.plot(maxcon[:,0],maxcon[:,1],'r', transform=ccrs.PlateCarree())
    #ax.plot([235,240],[48,48], transform=ccrs.PlateCarree())
    ax.set_extent([100, 260, 20, 65], crs=ccrs.PlateCarree())
    text=ax.text(225,22,np.datetime_as_string(ds2.time[itime], unit='D'),size=20,transform=ccrs.PlateCarree(),zorder=100)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlocator = mticker.FixedLocator([100,140,180,-140,-100])
    gl.ylocator = mticker.FixedLocator(np.arange(20,65,10))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15, 'color': 'black'}
    gl.ylabel_style = {'size': 15, 'color': 'black'}
    adir_figs='F:/data/NASA_biophysical/biforcation_figures_chelle/figs/'
    adir_data='F:/data/NASA_biophysical/biforcation_figures_chelle/'
    fname=adir_figs+str(np.datetime_as_string(ds2.time[itime], unit='D')).zfill(2)+'.png'
#    plt.show()
    print(fname)
    plt.savefig(fname) 
    plt.clf()
    maxcon2.append(maxcon)   
for i in range(ds2.time.size):
    maxcon=maxcon2[i]

    #find values u, ssh, etc... at -165 and -135, this will also save lat
    arr=np.abs(maxcon[:,0]-195)
    result = np.where(arr == np.amin(arr))[0][0]
    tem=ds2.isel(time=itime).sel(lon=maxcon[result,0],lat=maxcon[result,1],method='nearest')
    tem.load()   
    if i==0:
        ds_save165=tem
    else:
        ds_save165=xr.concat([ds_save165,tem],dim='time')

        #find values u, ssh, etc... at -165 and -135, this will also save lat
    arr=np.abs(maxcon[:,0]-225)
    result = np.where(arr == np.amin(arr))[0][0]
    tem=ds2.isel(time=itime).sel(lon=maxcon[result,0],lat=maxcon[result,1],method='nearest')
    tem.load()   
    if i==0:
        ds_save135=tem
    else:
        ds_save135=xr.concat([ds_save135,tem],dim='time')

    dlon=xr.DataArray(maxcon[:,0],dims=('z'),coords={'z':np.arange(len(maxcon))},name='lon')
    dlat=xr.DataArray(maxcon[:,1],dims=('z'),coords={'z':np.arange(len(maxcon))},name='lat')
    ds3=xr.Dataset({'lon':dlon,'lat':dlat})
    if i==0:
        ds4=ds3
    else:
        ds4=xr.concat([ds4,ds3],dim='time')
ds4['time']=ds2['time']
ds4.to_netcdf(adir_data+'biforcation_contour_quarterly_aviso.nc')
ds_save165.to_netcdf(adir_data+'biforcation_data165_quarterly_aviso.nc')
ds_save135.to_netcdf(adir_data+'biforcation_data135_quarterly_aviso.nc')

In [ ]:
maxcon2=[]
plt.figure(figsize=(15,6))
for itime in range(ds2.time.size):
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
    ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')
    ax.coastlines()
    X,Y=np.meshgrid(ds2.lon,ds2.lat)
    ds2 = ds2.sel(lon=slice(100,260),lat=slice(20,65))
    z=ds2.adt[itime,:,:]
    z3=data_smooth(z)
    z3[np.isnan(z)]=np.nan
    CS = ax.contour(X,Y,z3,30)
    num_levels = len(CS.allsegs)
    maxlat=40
    for ilev in range(num_levels):
        num_segments = len(CS.allsegs[ilev])
        index_save,jndex_save=[],[]
        for iseg in range(num_segments):
            num_pts = len(CS.allsegs[ilev][iseg])
            aseg = CS.allsegs[ilev][iseg]
            ax.plot(aseg[:,0],aseg[:,1],color='grey',transform=ccrs.PlateCarree())
            if itime==8 and max(aseg[:,1])>45:  #this is to fix a broken contour
                continue
            if itime==60 and aseg[1,1]>40:  #this is to fix a broken contour
                continue
            if itime==93 and aseg[1,1]>45:  #this is to fix a broken contour
                continue
            if itime==100 and aseg[1,1]>45:  #this is to fix a broken contour
                continue
            if min(aseg[:,0])>180 or max(aseg[:,1])>55:
                continue
            ind = (aseg[:,0]>235) & (aseg[:,0]<240) & (aseg[:,1]<48)
            if any(ind) & (aseg[:,1].max()<51):
                index_save.append(ilev)            
                jndex_save.append(iseg)            
            if any(ind) & any(aseg[:,1]>maxlat) & any(aseg[:,0]<230):
                maxlat=aseg[:,1].max().data
                maxcon=aseg
                print(aseg[1,:])
    ax.plot(maxcon[:,0],maxcon[:,1],'r', transform=ccrs.PlateCarree())
    #ax.plot([235,240],[48,48], transform=ccrs.PlateCarree())
    ax.set_extent([100, 260, 20, 65], crs=ccrs.PlateCarree())
    text=ax.text(225,22,np.datetime_as_string(ds2.time[itime], unit='D'),size=20,transform=ccrs.PlateCarree(),zorder=100)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlocator = mticker.FixedLocator([100,140,180,-140,-100])
    gl.ylocator = mticker.FixedLocator(np.arange(20,65,10))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15, 'color': 'black'}
    gl.ylabel_style = {'size': 15, 'color': 'black'}
    fname=adir_figs+str(np.datetime_as_string(ds2.time[itime], unit='D')).zfill(2)+'.png'
#    plt.show()
    print(fname)
    plt.savefig(fname) 
    plt.clf()
    maxcon2.append(maxcon)   
for i in range(ds2.time.size):
    maxcon=maxcon2[i]

    #find values u, ssh, etc... at -165 and -135, this will also save lat
    arr=np.abs(maxcon[:,0]-195)
    result = np.where(arr == np.amin(arr))[0][0]
    tem=ds2.isel(time=itime).sel(lon=maxcon[result,0],lat=maxcon[result,1],method='nearest')
    tem.load()   
    if i==0:
        ds_save165=tem
    else:
        ds_save165=xr.concat([ds_save165,tem],dim='time')

        #find values u, ssh, etc... at -165 and -135, this will also save lat
    arr=np.abs(maxcon[:,0]-225)
    result = np.where(arr == np.amin(arr))[0][0]
    tem=ds2.isel(time=itime).sel(lon=maxcon[result,0],lat=maxcon[result,1],method='nearest')
    tem.load()   
    if i==0:
        ds_save135=tem
    else:
        ds_save135=xr.concat([ds_save135,tem],dim='time')

    dlon=xr.DataArray(maxcon[:,0],dims=('z'),coords={'z':np.arange(len(maxcon))},name='lon')
    dlat=xr.DataArray(maxcon[:,1],dims=('z'),coords={'z':np.arange(len(maxcon))},name='lat')
    ds3=xr.Dataset({'lon':dlon,'lat':dlat})
    if i==0:
        ds4=ds3
    else:
        ds4=xr.concat([ds4,ds3],dim='time')
ds4['time']=ds2['time']
ds_save165['time']=ds2['time']
ds_save135['time']=ds2['time']
ds4.to_netcdf(adir_data+'biforcation_contour_quarterly_aviso.nc')
ds_save165.to_netcdf(adir_data+'biforcation_data165_quarterly_aviso.nc')
ds_save135.to_netcdf(adir_data+'biforcation_data135_quarterly_aviso.nc')

In [ ]:
ds2 = ds.resample(time='AS-JAN',keep_attrs=True).mean(skipna=False)
ds2 = ds2.sel(lon=slice(100,260),lat=slice(20,65))
X,Y=np.meshgrid(ds2.lon,ds2.lat)


In [ ]:
maxcon2=[]
plt.figure(figsize=(15,6))
for itime in range(ds2.time.size):
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
    ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')
    ax.coastlines()
    X,Y=np.meshgrid(ds2.lon,ds2.lat)
    ds2 = ds2.sel(lon=slice(100,260),lat=slice(20,65))
    z=ds2.adt[itime,:,:]
    z3=data_smooth(z)
    z3[np.isnan(z)]=np.nan
    CS = ax.contour(X,Y,z3,30)
    num_levels = len(CS.allsegs)
    maxlat=40
    for ilev in range(num_levels):
        num_segments = len(CS.allsegs[ilev])
        index_save,jndex_save=[],[]
        for iseg in range(num_segments):
            num_pts = len(CS.allsegs[ilev][iseg])
            aseg = CS.allsegs[ilev][iseg]
            ax.plot(aseg[:,0],aseg[:,1],color='grey',transform=ccrs.PlateCarree())
#            if itime==8 and max(aseg[:,1])>45:  #this is to fix a broken contour
#                continue
#            if itime==60 and aseg[1,1]>40:  #this is to fix a broken contour
#                continue
#            if itime==93 and aseg[1,1]>45:  #this is to fix a broken contour
#                continue
#            if itime==100 and aseg[1,1]>45:  #this is to fix a broken contour
#                continue
            if min(aseg[:,0])>180 or max(aseg[:,1])>55:
                continue
            ind = (aseg[:,0]>235) & (aseg[:,0]<240) & (aseg[:,1]<48)
            if any(ind) & (aseg[:,1].max()<51):
                index_save.append(ilev)            
                jndex_save.append(iseg)            
            if any(ind) & any(aseg[:,1]>maxlat) & any(aseg[:,0]<230):
                maxlat=aseg[:,1].max().data
                maxcon=aseg
                print(aseg[1,:])
    ax.plot(maxcon[:,0],maxcon[:,1],'r', transform=ccrs.PlateCarree())
    #ax.plot([235,240],[48,48], transform=ccrs.PlateCarree())
    ax.set_extent([100, 260, 20, 65], crs=ccrs.PlateCarree())
    text=ax.text(225,22,np.datetime_as_string(ds2.time[itime], unit='D'),size=20,transform=ccrs.PlateCarree(),zorder=100)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlocator = mticker.FixedLocator([100,140,180,-140,-100])
    gl.ylocator = mticker.FixedLocator(np.arange(20,65,10))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15, 'color': 'black'}
    gl.ylabel_style = {'size': 15, 'color': 'black'}

    fname=adir_figs+'/annual/'+str(np.datetime_as_string(ds2.time[itime], unit='D')).zfill(2)+'.png'
#    plt.show().
    print(fname)
    plt.savefig(fname) 
    plt.clf()
    maxcon2.append(maxcon)   
for i in range(ds2.time.size):
    maxcon=maxcon2[i]

    #find values u, ssh, etc... at -165 and -135, this will also save lat
    arr=np.abs(maxcon[:,0]-195)
    result = np.where(arr == np.amin(arr))[0][0]
    tem=ds2.isel(time=itime).sel(lon=maxcon[result,0],lat=maxcon[result,1],method='nearest')
    tem.load()   
    if i==0:
        ds_save165=tem
    else:
        ds_save165=xr.concat([ds_save165,tem],dim='time')

        #find values u, ssh, etc... at -165 and -135, this will also save lat
    arr=np.abs(maxcon[:,0]-225)
    result = np.where(arr == np.amin(arr))[0][0]
    tem=ds2.isel(time=itime).sel(lon=maxcon[result,0],lat=maxcon[result,1],method='nearest')
    tem.load()   
    if i==0:
        ds_save135=tem
    else:
        ds_save135=xr.concat([ds_save135,tem],dim='time')

    dlon=xr.DataArray(maxcon[:,0],dims=('z'),coords={'z':np.arange(len(maxcon))},name='lon')
    dlat=xr.DataArray(maxcon[:,1],dims=('z'),coords={'z':np.arange(len(maxcon))},name='lat')
    ds3=xr.Dataset({'lon':dlon,'lat':dlat})
    if i==0:
        ds4=ds3
    else:
        ds4=xr.concat([ds4,ds3],dim='time')
ds4['time']=ds2['time']
ds_save165['time']=ds2['time']
ds_save135['time']=ds2['time']
ds4.to_netcdf(dir_data+'biforcation_contour_annual_aviso.nc')
ds_save165.to_netcdf(dir_data+'biforcation_data165_annual_aviso.nc')
ds_save135.to_netcdf(dir_data+'biforcation_data135_annual_aviso.nc')

In [ ]:
#np.int(ds4.time[0].dt.year.data)
#colors[13]

In [ ]:
import matplotlib
from matplotlib import cm
from numpy import linspace
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
fig = plt.figure(figsize=(18,3.5))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
dir_figs='F:/data/NASA_biophysical/biforcation_figures_chelle/figs/'
start,stop,number_of_lines = 0.0,1.0,27
norm = matplotlib.colors.Normalize(vmin=1992,vmax=2020)
c_m = matplotlib.cm.jet
# create a ScalarMappable and initialize a data structure
s_m = plt.cm.ScalarMappable(cmap=c_m, norm=norm)
s_m.set_array([])
for i in range(27):
    ii=1992+i
    ax.plot(ds4.lon[i,:],ds4.lat[i,:],color=s_m.to_rgba(ii),transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_extent([130,250,30,40])
cb=plt.colorbar(s_m,fraction=.1)
fig.savefig(dir_figs + 'all_annual.png', dpi=300)


In [ ]:
#test annual data
tem = ds.adt[0:400,:,:].resample(time='AS-JAN',keep_attrs=True).mean(skipna=False)
print(tem[0,360,720].load())
tem2 = ds.adt[0:365,:,:].mean({'time'},skipna=False)
print(tem2[360,720].load())

In [ ]:
#test quarterly data
tem = ds.adt[300:450,:,:].resample(time='QS-DEC',keep_attrs=True).mean(skipna=False)
print(tem[0,360,720].load())
print(tem[1,360,720].load())
tem2 = ds.adt[334:424,:,:].mean({'time'},skipna=False)
print(tem2[360,720].load())

In [ ]:
print(ds.time[334].data)
ds.time[424].data

In [ ]:
import xarray as xr
ds=xr.open_dataset('./biforcation_data165_qu_aviso.nc')
ds

In [ ]:
ds.lat.plot()

In [ ]:
dir_data

In [ ]:
ds_save165.lat